In [1]:
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

c:\Users\rayas\OneDrive\Desktop\github\RMP-RAG\myenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
pc.create_index(
    name="rag-new",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


NameError: name 'pc' is not defined

In [5]:
# Load the review data
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Great professor! Very clear lectures and helpful during office hours.'},
 {'professor': 'Dr. Alice Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Good explanations, but sometimes a bit fast-paced.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Decent lectures, but not very engaging.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Chemistry',
  'stars': 2,
  'review': 'Material is hard, and lectures are confusing.'},
 {'professor': 'Dr. Linda Miller',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Very knowledgeable and approachable.'},
 {'professor': 'Dr. Richard Wilson',
  'subject': 'History',
  'stars': 5,
  'review': 'Passionate about the subject and makes history interesting.'},
 {'professor': 'Dr. Susan Lee',
  'subject': 'Literature',
  'stars': 4,
  'review': 'Engaging lectures, but the reading load is heavy.'},
 {'prof

In [7]:
processed_data = []
client = OpenAI()

# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable